# Second Term Project: CQADupStack Collection

The *CQADupStack* is "[a] Benchmark Data Set for Community Question-Answering Research" [1] that is a part of the [*Benchmarking Information Retrieval (BEIR)*](https://github.com/beir-cellar/beir) collection.

CQADupStack contains data from 12 different [*Stackexchange*](https://stackexchange.com/) subforums based on the data dump released on September 26, 2014.

Your tasks, reviewed by your colleagues and the course instructors, are the following:



1. *Implement a ranked retrieval system*, [1, Chapter 6] which will produce a list of documents from the CQADupStack collection in a descending order of relevance to a query from the CQADupStack collection.
2. *Document your code* in accordance with [PEP 257](https://www.python.org/dev/peps/pep-0257/), ideally using [the NumPy style guide](https://numpydoc.readthedocs.io/en/latest/format.html#docstring-standard) as seen in the code from exercises.
   *Stick to a consistent coding style* in accordance with [PEP 8](https://www.python.org/dev/peps/pep-0008/).
3. *Reach at least 25% mean average precision at 10* [1, Section 8.4] with your system on the CQADupStack collection.
4. _[Upload an .ipynb file](https://is.muni.cz/help/komunikace/spravcesouboru#k_ss_1) with this Jupyter notebook to the homework vault in IS MU._ You MAY also include a brief description of your information retrieval system and a link to an external service such as [Google Colaboratory](https://colab.research.google.com/), [DeepNote](https://deepnote.com/), or [JupyterHub](https://iirhub.cloud.e-infra.cz/).







[1] Hoogeveen, Doris and Verspoor, Karin M. and Baldwin, Timothy. [*CQADupStack: A Benchmark Data Set for Community Question-Answering Research*](https://dl.acm.org/doi/10.1145/2838931.2838934). ACM, 2015.

## Import the utility tools from the git repository.

First, we will install [our library](https://github.com/MIR-MU/pv211-utils).

It may be necessary to restart the runtime to get the installed packages to work.

In [ ]:
%%capture
! pip install git+https://github.com/MIR-MU/pv211-utils.git

## Define the necessary classes

These will eventually represent the Queries, Documents and Relevance Judgements from the CQADupStack collection.

Query and Document consist only of their IDs and bodies.
Judgements are also just a Set of Tuples that represent pairs of relevant Document-Query combinations.

In [ ]:
from pv211_utils.beir.entities import BeirDocumentBase, BeirQueryBase, BeirJudgementBase
from typing import Set


class Query(BeirQueryBase):
    """
    A processed query form the Beir collection.

    Parameters
    ----------
    query_id : int
        The number
    body : str
        Text of a query

    """

    def __init__(self, query_id: int, body: str):
        super().__init__(query_id, body)

    def __str__(self):
        return self.body


class Document(BeirDocumentBase):
    """
    A processed document form the Beir collection.

    Parameters
    ----------
    document_id : str
        A unique identifier of the document.
    body : str
        The text of the document.

    """

    def __init__(self, document_id: str, body: str):
        super().__init__(document_id, body)

    def __str__(self):
        return self.body


BeirJudgements = Set[BeirJudgementBase]


## Loading the datasets
### CQADupStack contains 12 datasets that will be loaded and merged:
- Android
- English
- Gaming
- GIS
- Mathematica
- Physics
- Programmers
- Stats
- TeX
- Unix
- Webmasters
- WordPress

For more details: <a href=http://nlp.cis.unimelb.edu.au/resources/cqadupstack/>CQADupStack site</a>.



In [ ]:
from pv211_utils.datasets import CQADupStackDataset 

data = CQADupStackDataset(validation_split_size=0.2)

In [ ]:
documents = data.load_documents(document_class=Document)

train_queries = data.load_train_queries(query_class=Query)
train_judgements = data.load_train_judgements()

validation_queries = data.load_validation_queries(query_class=Query)
validation_judgements = data.load_validation_judgements()

## Implementation of information retrieval system

Here we will define our IR system. If you want to use your own class it must define a method name `search` that takes a query and returns documents in descending order of relevance to the query.

This example returns documents in a decreasing order according to
a [*Okapi BestMatch25+*](https://en.wikipedia.org/wiki/Okapi_BM25#Modifications) similarity score between the documents and the given query.

If you wish you might use [preprocessing](https://github.com/MIR-MU/pv211-utils/tree/main/pv211_utils/preprocessing) or [ensemble](https://github.com/MIR-MU/pv211-utils/blob/developer/pv211_utils/ensembles.py) techniques from our library.

In [ ]:
# Move the processing to the cuda

import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

In [ ]:
from gensim.parsing.porter import PorterStemmer
from gensim.utils import simple_preprocess
from nltk.stem import WordNetLemmatizer
from typing import Union
from nltk.tokenize import word_tokenize
import nltk
nltk.download('wordnet')
nltk.download('punkt')

def preprocess(document):
    """The preprocessing of a given document, which removes stopwords and does the stemming.

        Parameters
        ----------
        document : Document
            The instance of Document class.
        
        Returns
        -------
        tokens : List[str]
            The list of tokens of a given document, retrieved by applying various preprocessing techniques.
            
    """
    stemmer = PorterStemmer()
    gist_file = open("gist_stopwords.txt", "r")
    try:
      content = gist_file.read()
      stopwords = content.split(",")
    finally:
      gist_file.close()
    tokens = [stemmer.stem(item) for item in word_tokenize(document) if not item in stopwords]

    return " ".join(tokens)

In [ ]:
from typing import Iterable
import numpy as np

from sentence_transformers import InputExample
from sentence_transformers.losses import MultipleNegativesRankingLoss
from torch.utils.data import DataLoader

from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import dot_score
from nltk.stem import WordNetLemmatizer


class IRSystem():
    """
    A system that returns documents ordered by decreasing cosine similarity, using the predictions of finetuned transformer model.

    Attributes
    ----------
    document_embeddings: 
        The dictionary of the system.
    transformer: SentenceTransformer
        The indexed documents.
    index_to_document: dict of (int, Document)
        A mapping from indexed document numbers to documents.

    """

    def __init__(self):
        # Generate index_to_document dictionary
        index_to_document = dict(enumerate(documents.values()))
        self.index_to_document = index_to_document
        
        # Download pretrained model from Huggingface
        transformer = SentenceTransformer("all-mpnet-base-v2").to(device)

        # Define the train examples, cropped to the length of 512 due to input length limitation of transformer.
        train_examples = [ InputExample(texts=[preprocess(judgement[0].body), 
                                                preprocess(judgement[1].body[:512])], 
                                        label=1.) for judgement in train_judgements
                          ]

        # Define your train dataset and the dataloader:
        train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

        # Finally, pick the objective for training (i.e. loss):
        train_loss = MultipleNegativesRankingLoss(transformer)

        # Tune the model
        transformer.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)

        # Generate embeddings from document bodies
        document_bodies = [preprocess(document.body) for document in documents.values()]
        document_embeddings = []
        for index in tqdm(range(0, len(document_bodies), 1028), desc='Dataset'):
            document_embeddings_b = transformer.encode(document_bodies[index:min(index+1200, len(document_bodies))])
            document_embeddings.extend(document_embeddings_b)
        
        self.document_embeddings = document_embeddings
        self.transformer = transformer

    def search(self, query: Query) -> Iterable[Document]:
        """The ranked retrieval results for a query.

        Parameters
        ----------
        query : Query
            A query.

        Returns
        -------
        iterable of Document
            The ranked retrieval results for a query.

        """

        # Calculate list of similarities using the transformer model
        query_vector_semantic = self.transformer.encode(preprocess(query.body))
        similarities_semantic = [
            (index, np.dot(query_vector_semantic, doc_vector))
            for index, doc_vector in enumerate(self.document_embeddings)
        ]

        # Combine similarities to calculate final list
        similarities = sorted((similarities_semantic), key=lambda item: item[1], reverse=True)

        for (document_number, _) in similarities:
            document = self.index_to_document[document_number]
            yield document

In [ ]:
from pv211_utils.systems import BM25PlusSystem, TfidfSystem
from pv211_utils.preprocessing import SimpleDocPreprocessing

system = IRSystem()

## Evaluate the system on a given dataset

We will evaluate the IR system using the [Mean Average Precision](https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)#Mean_average_precision) (MAP).

In [ ]:
from pv211_utils.beir.leaderboard import BeirLeaderboard
from pv211_utils.beir.eval import BeirEvaluation

submit_result = False
author_name = ""

test_queries = data.load_test_queries(Query)
test_judgements = data.load_test_judgements()
leaderboard = BeirLeaderboard()
evaluation = BeirEvaluation(system, test_judgements, k=10, leaderboard=leaderboard, author_name=author_name)
evaluation.evaluate(test_queries, submit_result)